    %reload_ext XXX.monkey
    ---
    %%monkey file -- args
    trace

In [1]:
from monkeytype.tracing import CallTraceLogger, trace_calls
from monkeytype.stubs import build_module_stubs_from_traces
DUNDER = '__%s__'

In [2]:
class IPythonTraceLogger(CallTraceLogger):
    """A CallTraceLogger that stores logged traces in a CallTraceStore."""
    def __init__(self, data=None) -> None:
        self.data = None or []
        self.traces = []

    def log(self, trace) -> None:
        self.traces.append(trace)

    def flush(self) -> None:
        self.data.extend(self.traces)
        self.traces = []
        
    def stubs(logger): return build_module_stubs_from_traces(logger.data).get(DUNDER%'main').render()

In [3]:
from contextlib import contextmanager

In [4]:
@contextmanager
def trace(logger=None):
    logger = logger or IPythonTraceLogger()
    with trace_calls(logger):
        yield logger
        return None

In [5]:
def monkey(line, cell):
    ip=__import__('IPython').get_ipython() 
    with trace() as logger:  
        eval(cell, ip.user_ns, ip.user_ns)
       
    if line: 
        if 'retype' in line:
            ip.set_next_input(retype(logger))
        else:
            ip.magics_manager.magics['cell']['file'](line, logger.stubs() + '\n')
    else:  
        ip.set_next_input(logger.stubs())

In [6]:
def retype(logger): 
    import retype
    from typed_ast import ast3
    from inspect import getsource
    src = '\n'.join(map(getsource, set(_.func for _ in logger.data if hasattr(_, 'func'))))
    retype.Config.incremental = False
    retype.Config.replace_any = True
    src = retype.lib2to3_parse(src)
    retype.reapply(
        ast3.parse(logger.stubs()).body, src)
    retype.fix_remaining_type_comments(src)
    return retype.lib2to3_unparse(src, hg=False)

In [7]:
def load_ipython_extension(ip=__import__('IPython').get_ipython()):
    ip.register_magic_function(monkey, 'cell')